<a href="https://colab.research.google.com/github/Praxis-QR/BDSN/blob/main/KK_C4_Spark_RDD_Operations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

Apache Spark: An Introduction
https://www.dataquest.io/blog/apache-spark/

In [1]:
from datetime import datetime
import pytz
print('ॐ श्री सरस्वत्यै नमः',datetime.now(pytz.timezone('Asia/Calcutta')))

!python --version
!lsb_release -a
#!cat /proc/cpuinfo
#
!cat /proc/cpuinfo |grep "physical id"|sort |uniq|wc -l # physical processor
!cat /proc/cpuinfo |grep "processor"|wc -l              # virtual processor

ॐ श्री सरस्वत्यै नमः 2024-04-24 10:40:09.704619+05:30
Python 3.10.12
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.3 LTS
Release:	22.04
Codename:	jammy
1
2


# Install Spark

In [2]:
import pandas as pd
#
!pip3 -q install pyspark
from pyspark.sql import SparkSession
spark51 = SparkSession.\
    builder.\
    master("local[*]").\
    appName("Yantrajaal"). \
    getOrCreate()
sc51 = spark51.sparkContext
sc51

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


<SparkContext master=local[*] appName=Yantrajaal>

# Daily Show Data Analysis

## Get DailyShowData

In [3]:
!wget -q  'https://raw.githubusercontent.com/Praxis-QR/BDSN/main/data/dailyshow/daily_show_guests.csv'

In [4]:
#
# non Pythonic, non Spark of doing things
#
import csv

f = open('daily_show_guests.csv')
tally_D = dict()
for line in f:
    data_line = line.split(',')
    year = data_line[0]
    if year in tally_D.keys():
       tally_D[year] = tally_D[year] + 1
    else:
       tally_D[year] = 1

print(tally_D)

{'YEAR': 1, '1999': 166, '2000': 169, '2001': 157, '2002': 159, '2003': 166, '2004': 164, '2005': 162, '2006': 161, '2007': 141, '2008': 164, '2009': 163, '2010': 165, '2011': 163, '2012': 164, '2013': 166, '2014': 163, '2015': 100}


## Introducing Spark

In [5]:
raw_data = sc51.textFile("daily_show_guests.csv")
raw_data.take(5)

['YEAR,GoogleKnowlege_Occupation,Show,Group,Raw_Guest_List',
 '1999,actor,1/11/99,Acting,Michael J. Fox',
 '1999,Comedian,1/12/99,Comedy,Sandra Bernhard',
 '1999,television actress,1/13/99,Acting,Tracey Ullman',
 '1999,film actress,1/14/99,Acting,Gillian Anderson']

In [6]:
daily_show = raw_data.map(lambda line: line.split(','))
daily_show.take(5)

[['YEAR', 'GoogleKnowlege_Occupation', 'Show', 'Group', 'Raw_Guest_List'],
 ['1999', 'actor', '1/11/99', 'Acting', 'Michael J. Fox'],
 ['1999', 'Comedian', '1/12/99', 'Comedy', 'Sandra Bernhard'],
 ['1999', 'television actress', '1/13/99', 'Acting', 'Tracey Ullman'],
 ['1999', 'film actress', '1/14/99', 'Acting', 'Gillian Anderson']]

In [7]:
tallyMap = daily_show.map(lambda x: (x[0], 1))
tallyMap.take(5)


[('YEAR', 1), ('1999', 1), ('1999', 1), ('1999', 1), ('1999', 1)]

In [8]:
tallyRed = tallyMap.reduceByKey(lambda x,y: x+y)
tallyRed.take(5)

[('YEAR', 1), ('2002', 159), ('2003', 166), ('2004', 164), ('2007', 141)]

In [9]:
tally = daily_show.map(lambda x: (x[0], 1)).reduceByKey(lambda x,y: x+y)
print(tally)

PythonRDD[14] at RDD at PythonRDD.scala:53


In [10]:
tally.count()

18

In [11]:
tally.take(tally.count())

[('YEAR', 1),
 ('2002', 159),
 ('2003', 166),
 ('2004', 164),
 ('2007', 141),
 ('2010', 165),
 ('2011', 163),
 ('2012', 164),
 ('2013', 166),
 ('2014', 163),
 ('2015', 100),
 ('1999', 166),
 ('2000', 169),
 ('2001', 157),
 ('2005', 162),
 ('2006', 161),
 ('2008', 164),
 ('2009', 163)]

In [12]:
def filter_year(line):
    if line[0] == 'YEAR':
        return False
    else:
        return True
filtered_daily_show = daily_show.filter(lambda line: filter_year(line))

In [13]:
tally2 = filtered_daily_show.map(lambda x: (x[0], 1)).reduceByKey(lambda x,y: x+y)
print(tally2)

PythonRDD[23] at RDD at PythonRDD.scala:53


In [14]:
tally2.take(tally2.count())

[('2002', 159),
 ('2003', 166),
 ('2004', 164),
 ('2007', 141),
 ('2010', 165),
 ('2011', 163),
 ('2012', 164),
 ('2013', 166),
 ('2014', 163),
 ('2015', 100),
 ('1999', 166),
 ('2000', 169),
 ('2001', 157),
 ('2005', 162),
 ('2006', 161),
 ('2008', 164),
 ('2009', 163)]

In [15]:
filtered_daily_show.filter(lambda line: line[1] != '') \
                   .map(lambda line: (line[1].lower(), 1)) \
                   .reduceByKey(lambda x,y: x+y) \
                   .take(20)

[('actor', 596),
 ('film actress', 21),
 ('model', 9),
 ('stand-up comedian', 44),
 ('actress', 271),
 ('television personality', 13),
 ('comic', 2),
 ('musician', 19),
 ('film actor', 19),
 ('journalist', 253),
 ('us senator', 50),
 ('vocalist', 2),
 ('film director', 24),
 ('singer', 23),
 ('muppet', 2),
 ('american television personality', 3),
 ('rapper', 10),
 ('football player', 1),
 ('former mayor of cincinatti', 2),
 ('businesswoman', 2)]

## Extra Sorting

In [16]:
filtered_daily_show.filter(lambda line: line[1] != '') \
                   .map(lambda line: (line[1].lower(), 1)) \
                   .reduceByKey(lambda x,y: x+y) \
                   .take(20)

[('actor', 596),
 ('film actress', 21),
 ('model', 9),
 ('stand-up comedian', 44),
 ('actress', 271),
 ('television personality', 13),
 ('comic', 2),
 ('musician', 19),
 ('film actor', 19),
 ('journalist', 253),
 ('us senator', 50),
 ('vocalist', 2),
 ('film director', 24),
 ('singer', 23),
 ('muppet', 2),
 ('american television personality', 3),
 ('rapper', 10),
 ('football player', 1),
 ('former mayor of cincinatti', 2),
 ('businesswoman', 2)]

In [17]:
#sorted by profession
#not sorted by number of appearances
# in sortbykey, 1st parameter determines direction of sort, second parameter is optional)
# in sortbykey, 2nd parameter determines number of partition in output RDD
# see next two cells
filtered_daily_show.filter(lambda line: line[1] != '') \
                   .map(lambda line: (line[1].lower(), 1)) \
                   .reduceByKey(lambda x,y: x+y) \
                   .sortByKey(0, 1) \
                   .take(20)

[('writer', 52),
 ('white house sommunications director', 5),
 ('white house official', 1),
 ('vocalist', 2),
 ('vice president', 2),
 ('us senator', 50),
 ('us secretary of energy', 1),
 ('us secretary of defense', 1),
 ('us secetary of education', 1),
 ('us representative', 9),
 ('us president', 10),
 ('us permanent representative to nato', 1),
 ('us official', 1),
 ('us assistant attorney', 1),
 ('university professor', 2),
 ('united states senator', 15),
 ('united states senate member', 4),
 ('united states secretary of the navy', 1),
 ('united states secretary of housing and urban development', 1),
 ('united states secretary of defense', 1)]

In [18]:
#sorted by profession
#not sorted by number of appearances
# in sortbykey, 1st parameter determines direction of sort, second parameter is optional)
# in sortbykey, 2nd parameter determines number of partition in output RDD
# see next two cells
filtered_daily_show.filter(lambda line: line[1] != '') \
                   .map(lambda line: (line[1].lower(), 1)) \
                   .reduceByKey(lambda x,y: x+y) \
                   .sortByKey(1, 1) \
                   .take(20)

[('-', 1),
 ('0', 4),
 ('academic', 6),
 ('accountant', 1),
 ('activist', 14),
 ('actor', 596),
 ('actress', 271),
 ('administrator', 1),
 ('admiral', 2),
 ('adviser', 4),
 ('advocate', 1),
 ('aei president', 1),
 ('afghan politician', 1),
 ('american football quarterback', 3),
 ('american football running back', 1),
 ('american football wide receiver', 2),
 ('american football wide reciever', 1),
 ('american political figure', 2),
 ('american politician', 8),
 ('american television personality', 3)]

In [19]:
!rm -r dumpDir
filtered_daily_show.filter(lambda line: line[1] != '') \
                   .map(lambda line: (line[1].lower(), 1)) \
                   .reduceByKey(lambda x,y: x+y) \
                   .sortByKey(0,2)\
                   .saveAsTextFile("dumpDir")

rm: cannot remove 'dumpDir': No such file or directory


In [20]:
# The original output
#
filtered_daily_show.filter(lambda line: line[1] != '') \
                   .map(lambda line: (line[1].lower(), 1)) \
                   .reduceByKey(lambda x,y: x+y) \
                   .take(10)

[('actor', 596),
 ('film actress', 21),
 ('model', 9),
 ('stand-up comedian', 44),
 ('actress', 271),
 ('television personality', 13),
 ('comic', 2),
 ('musician', 19),
 ('film actor', 19),
 ('journalist', 253)]

In [21]:
# Flip the key and value
#
filtered_daily_show.filter(lambda line: line[1] != '') \
                   .map(lambda line: (line[1].lower(), 1)) \
                   .reduceByKey(lambda x,y: x+y) \
                   .map(lambda x: (x[1],x[0]))\
                   .take(10)

[(596, 'actor'),
 (21, 'film actress'),
 (9, 'model'),
 (44, 'stand-up comedian'),
 (271, 'actress'),
 (13, 'television personality'),
 (2, 'comic'),
 (19, 'musician'),
 (19, 'film actor'),
 (253, 'journalist')]

In [22]:
# Flip the key and value
#
filtered_daily_show.filter(lambda line: line[1] != '') \
                   .map(lambda line: (line[1].lower(), 1)) \
                   .reduceByKey(lambda x,y: x+y) \
                   .map(lambda x: (x[1],x[0]))\
                   .sortByKey(0, 1) \
                   .take(10)

[(596, 'actor'),
 (271, 'actress'),
 (253, 'journalist'),
 (152, 'author'),
 (103, 'comedian'),
 (52, 'writer'),
 (50, 'us senator'),
 (44, 'stand-up comedian'),
 (39, 'television host'),
 (37, 'professor')]

In [23]:
#to sort by number of appearances, we first need to flip the key and value
#

# sort 1st arg configures ascending sort, 2nd configures 1 task
# https://stackoverflow.com/questions/38569052/how-do-i-invert-key-and-value-in-rdd-in-python-3-pyspark

filtered_daily_show.filter(lambda line: line[1] != '') \
                   .map(lambda line: (line[1].lower(), 1)) \
                   .reduceByKey(lambda x,y: x+y) \
                   .map(lambda x: (x[1],x[0]))\
                   .sortByKey(0, 1) \
                   .map(lambda x: (x[1],x[0])) \
                   .take(10)

[('actor', 596),
 ('actress', 271),
 ('journalist', 253),
 ('author', 152),
 ('comedian', 103),
 ('writer', 52),
 ('us senator', 50),
 ('stand-up comedian', 44),
 ('television host', 39),
 ('professor', 37)]

In [24]:
topTen = \
filtered_daily_show.filter(lambda line: line[1] != '') \
                   .map(lambda line: (line[1].lower(), 1)) \
                   .reduceByKey(lambda x,y: x+y) \
                   .map(lambda x: (x[1],x[0]))\
                   .sortByKey(0, 1) \
                   .map(lambda x: (x[1],x[0])) \
                   .take(10)

In [25]:
for t in topTen:
    print(t[0],t[1])

actor 596
actress 271
journalist 253
author 152
comedian 103
writer 52
us senator 50
stand-up comedian 44
television host 39
professor 37


## Other Sort Methods

In [26]:
filtered_daily_show.filter(lambda line: line[1] != '') \
                   .map(lambda line: (line[1].lower(), 1)) \
                   .reduceByKey(lambda x,y: x+y) \
                   .sortBy(lambda x : -1*x[1])\
                   .take(10)

[('actor', 596),
 ('actress', 271),
 ('journalist', 253),
 ('author', 152),
 ('comedian', 103),
 ('writer', 52),
 ('us senator', 50),
 ('stand-up comedian', 44),
 ('television host', 39),
 ('professor', 37)]

In [27]:
filtered_daily_show.filter(lambda line: line[1] != '') \
                   .map(lambda line: (line[1].lower(), 1)) \
                   .reduceByKey(lambda x,y: x+y) \
                   .takeOrdered(10, lambda aTuple:-1*aTuple[1])

[('actor', 596),
 ('actress', 271),
 ('journalist', 253),
 ('author', 152),
 ('comedian', 103),
 ('writer', 52),
 ('us senator', 50),
 ('stand-up comedian', 44),
 ('television host', 39),
 ('professor', 37)]

# Climate Trace Emissions Data

In [28]:
!rm *.csv
# Climate Trace Data stored in github
!wget -O firesData.csv -q  'https://github.com/Praxis-QR/BDSN/raw/main/Data2/cropland-fires_country_emissions.csv'
!wget -O feedData.csv  -q 'https://github.com/Praxis-QR/BDSN/raw/main/Data2/enteric-fermentation-cattle-feedlot_country_emissions.csv'
!wget -O pastureData.csv -q  'https://github.com/Praxis-QR/BDSN/raw/main/Data2/enteric-fermentation-cattle-pasture_country_emissions.csv'

In [29]:
baseData = sc51.textFile("/content/feedData.csv")
feed = baseData.count()
baseData = sc51.textFile("/content/firesData.csv")
fires = baseData.count()
baseData = sc51.textFile("/content/pastureData.csv")
pastures = baseData.count()
print(feed, fires, pastures)

10001 8241 6801


In [30]:
baseData = sc51.textFile("/content/*.csv")
baseData.count()

25043

In [31]:
baseData.take(5)

['iso3_country,start_time,end_time,original_inventory_sector,gas,emissions_quantity,emissions_quantity_units,temporal_granularity,created_date,modified_date',
 'GNB,2015-01-01 00:00:00,2015-12-31 00:00:00,enteric-fermentation-cattle-feedlot,co2,0,tonnes,annual,2023-10-03 21:10:33.91199,',
 'AFG,2015-01-01 00:00:00,2015-12-31 00:00:00,enteric-fermentation-cattle-feedlot,co2,0,tonnes,annual,2023-10-03 21:10:33.871002,',
 'AFG,2015-01-01 00:00:00,2015-12-31 00:00:00,enteric-fermentation-cattle-feedlot,ch4,270283.3,tonnes,annual,2023-10-03 21:10:33.871018,',
 'AFG,2015-01-01 00:00:00,2015-12-31 00:00:00,enteric-fermentation-cattle-feedlot,n2o,0,tonnes,annual,2023-10-03 21:10:33.871027,']

# Climate Trace Emissions Data

## Remove Headers

In [32]:
baseData.filter(lambda x: 'iso3_country' in x).collect()

['iso3_country,start_time,end_time,original_inventory_sector,gas,emissions_quantity,emissions_quantity_units,temporal_granularity,created_date,modified_date',
 'iso3_country,start_time,end_time,original_inventory_sector,gas,emissions_quantity,emissions_quantity_units,temporal_granularity,created_date,modified_date',
 'iso3_country,start_time,end_time,original_inventory_sector,gas,emissions_quantity,emissions_quantity_units,temporal_granularity,created_date,modified_date']

In [33]:
data2 = baseData.filter(lambda x: 'iso3_country' not in x)
data2.count()

25040

## Basic Parsing

In [34]:
def parse01(text):
    e = text.split(',')
    return(e[0], e[1][:4],e[4],float(e[5]))

data2.map(lambda x : parse01(x)).take(5)

[('GNB', '2015', 'co2', 0.0),
 ('AFG', '2015', 'co2', 0.0),
 ('AFG', '2015', 'ch4', 270283.3),
 ('AFG', '2015', 'n2o', 0.0),
 ('AFG', '2015', 'co2e_100yr', 7351705.7)]

## Simple Map Reduce

In [35]:
def year_gas(text):
    try:
        e = text.split(',')
        key = e[1][:4]+'-'+e[4]
        val = float(e[5])/1000000
        return(key,val)
    except:
        return('dummy',0)

#data2.map(lambda x : year_gas(x)).take(5)
data2.map(lambda x : year_gas(x)).reduceByKey(lambda x,y:x+y).sortByKey().collect()

[('2015-ch4', 75.50691078105656),
 ('2015-co2', 1046.9770209610153),
 ('2015-co2e_100yr', 3113.9714169259696),
 ('2015-co2e_20yr', 7161.1418350306),
 ('2015-n2o', 0.048375175238883676),
 ('2016-ch4', 76.47741725534249),
 ('2016-co2', 1057.6678797667494),
 ('2016-co2e_100yr', 3151.194905036182),
 ('2016-co2e_20yr', 7250.384470442539),
 ('2016-n2o', 0.0488691435315678),
 ('2017-ch4', 76.79535368702778),
 ('2017-co2', 1103.1228447425897),
 ('2017-co2e_100yr', 3205.8711031696303),
 ('2017-co2e_20yr', 7322.102061754316),
 ('2017-n2o', 0.050969372234014015),
 ('2018-ch4', 77.57481713237699),
 ('2018-co2', 1096.2346056464169),
 ('2018-co2e_100yr', 3220.097383540456),
 ('2018-co2e_20yr', 7378.107581235868),
 ('2018-n2o', 0.0506511047376751),
 ('2019-ch4', 78.40915454114136),
 ('2019-co2', 1090.8370552232002),
 ('2019-co2e_100yr', 3237.3257262002094),
 ('2019-co2e_20yr', 7440.056409325387),
 ('2019-n2o', 0.05040171178741476),
 ('2020-ch4', 79.28938240211528),
 ('2020-co2', 1101.9819522788177),


In [36]:
def year_gas_error(text):
    try:
        e = text.split(',')
        key = e[1][:4]+'-'+e[4]
        val = float(e[5])
        return(False)
    except:
        return(True)

#data2.filter(lambda x: year_gas_error(x)).take(5)
data2.filter(lambda x: year_gas_error(x)).count()

2688

In [37]:
#Chained Pipeline
#
ygl = sc51.textFile("/content/*.csv")\
        .filter(lambda x: 'iso3_country' not in x)\
        .map(lambda x : year_gas(x))\
        .reduceByKey(lambda x,y:x+y)\
        .sortByKey()\
        .collect()

In [38]:
emissions_df = pd.DataFrame(ygl,columns =['Type', 'Tonnes'],)
emissions_df

,Type,Tonnes
0,2015-ch4,75.506911
1,2015-co2,1046.977021
2,2015-co2e_100yr,3113.971417
3,2015-co2e_20yr,7161.141835
4,2015-n2o,0.048375
5,2016-ch4,76.477417
6,2016-co2,1057.667880
7,2016-co2e_100yr,3151.194905
8,2016-co2e_20yr,7250.384470
9,2016-n2o,0.048869


In [39]:
emissions_df[['Year', 'GasType']] = emissions_df['Type'].str.split('-', expand=True)
emissions_df

,Type,Tonnes,Year,GasType
0,2015-ch4,75.506911,2015,ch4
1,2015-co2,1046.977021,2015,co2
2,2015-co2e_100yr,3113.971417,2015,co2e_100yr
3,2015-co2e_20yr,7161.141835,2015,co2e_20yr
4,2015-n2o,0.048375,2015,n2o
5,2016-ch4,76.477417,2016,ch4
6,2016-co2,1057.667880,2016,co2
7,2016-co2e_100yr,3151.194905,2016,co2e_100yr
8,2016-co2e_20yr,7250.384470,2016,co2e_20yr
9,2016-n2o,0.048869,2016,n2o


#Chronobooks <br>
Two science fiction novels by Prithwis Mukerjee. A dystopian Earth. A technocratic society managed by artificial intelligence. Escape and epiphany on Mars. Can man and machine, carbon and silicon explore and escape into other dimensions of existence? An Indic perspective rooted in Advaita Vedanta and the Divine Feminine.  [More information](http://bit.ly/chronobooks) <br>
![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/CTCYFooter-1.png)

In [40]:
from datetime import datetime
import pytz
print('signed off at  ',datetime.now(pytz.timezone('Asia/Kolkata')))

signed off at   2024-04-24 10:42:01.517237+05:30
